In [130]:
import sys
import json

params = {'general':{'period':'test'}}
default = {'training_folder': '../..', 'params': params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../..', 'params': {'general': {'period': 'test'}}}


In [127]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel


In [129]:
period = argv['params']['general']['period']
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
print('On Lambda : ', on_lambda)
training_folder = argv['training_folder']
input_folder = training_folder +r'/inputs/'

if on_lambda:
    scenario_folder = training_folder +r'/inputs/'
    output_folder = training_folder + '/outputs/'
    model_folder = training_folder + '/model/'
else:
    scenario_folder = training_folder + '/scenarios/' + period + '/inputs/'
    output_folder = training_folder + '/scenarios/' + period + '/outputs/'
    model_folder = training_folder + '/scenarios/' + period + '/model/'
print('input folder: ', input_folder)
print('output folder: ', output_folder)
print('scen folder : ', scenario_folder)
print('model folder : ', model_folder)


pt_folder = scenario_folder + 'pt/'
road_folder = scenario_folder + 'road/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

On Lambda :  False
input folder:  ../../inputs/
output folder:  ../../scenarios/test/outputs/
scen folder :  ../../scenarios/test/inputs/
model folder :  ../../scenarios/test/model/


In [89]:
var, ancestry = excel.read_var(file=input_folder+'/parameters.xlsx', scenario=period, return_ancestry=True)


# inputs

In [90]:
sm = stepmodel.read_zippedpickles(model_folder + 'network')

road_nodes: 100%|███████████████████████████████| 10/10 [00:00<00:00, 11.39it/s]


In [133]:
filepath = excel.get_filepath(input_folder + '{s}/volumes/volumes.csv', ancestry=ancestry, log=False)
volumes = pd.read_csv(filepath)
volumes = volumes.set_index('index')

In [136]:
# restrict to mtl
#sm.zones = sm.zones[sm.zones['SM']<200]
#zones_list = sm.zones.index.values
#volumes = volumes[(volumes['origin'].isin(zones_list)) & (volumes['destination'].isin(zones_list)) ]


# pathfinder

In [137]:
od_set = set(zip(volumes['origin'], volumes['destination']))


In [138]:
sm.step_pt_pathfinder(
    broken_routes=False,
    broken_modes=False,
    keep_pathfinder=True,
    mode_column='route_type',
    route_column='route_id',
    speedup=True,
    walk_on_road=False,
    path_analysis=False,
    od_set=od_set,
    engine='b', # b is faster
)
sm.pt_los['time'] = sm.pt_los['gtime']

start publicpathfinder
path_analysis


# car pathfinder

In [139]:
car = sm.copy()
car.road_links['dup'] = car.road_links['a'] + car.road_links['b']
car.road_links = car.road_links.drop_duplicates('dup').drop(columns=['dup'])

car.preparation_ntlegs(
    short_leg_speed=15,
    long_leg_speed=15,
    threshold=5000,
    zone_to_transit=False,
    zone_to_road=True,
    n_ntlegs=1,
)
car.step_road_pathfinder(method='aon', access_time='time')

In [140]:
car.car_los[['origin','destination','time']].to_csv(scenario_folder +'car_skim.csv')
sm.pt_los[['origin','destination','time']].to_csv(scenario_folder +'pt_skim.csv')

In [141]:
car.car_los[['origin','destination','time']].to_csv(output_folder +'car_skim.csv')
sm.pt_los[['origin','destination','time']].to_csv(output_folder +'pt_skim.csv')

In [142]:
output_folder

'../../scenarios/test/outputs/'

In [143]:
car.car_los[['origin','destination','time']]

,origin,destination,time
index,,,
0,zone_1,zone_278,648.765203
1,zone_1,zone_298,1013.960854
2,zone_1,zone_352,1048.322510
3,zone_1,zone_513,1534.511044
4,zone_1,zone_565,1029.064867
...,...,...,...
44572,zone_98,zone_61,260.294903
44573,zone_98,zone_69,283.182814
44574,zone_98,zone_691,640.517417


In [144]:
sm.pt_los[['origin','destination','time']]

,origin,destination,time
0,zone_1,zone_278,2184.221764
1,zone_1,zone_298,3816.574616
2,zone_1,zone_352,2989.997330
3,zone_1,zone_513,4287.612792
4,zone_1,zone_565,3678.755931
...,...,...,...
19885,zone_98,zone_61,2097.355637
19886,zone_98,zone_69,1769.735518
19887,zone_98,zone_691,1773.902373
19888,zone_98,zone_694,2195.467669


In [145]:
sm.car_los = car.car_los

In [146]:
sm.to_zippedpickles(model_folder +'los', omitted_attributes=['publicpathfinder'])

car_los: 100%|██████████████████████████████████| 18/18 [00:03<00:00,  5.62it/s]
